In [ ]:
%load_ext autoreload

%autoreload 2

import zarr
from oxeo.water.models.pekel import utils
from oxeo.water.models.pekel import masks
from oxeo.water.models.pekel.pekel import PekelPredictor
import attr
import matplotlib.pyplot as plt
import dask.array as da

import gcsfs
fs = gcsfs.GCSFileSystem(project='oxeo-main')


In [ ]:
patches_paths = fs.ls('oxeo-water/eo/hoover')
patches_paths

In [ ]:
predictor = PekelPredictor()

In [ ]:
for p_path in patches_paths:
    arr = zarr.open(f"gs://{p_path}/sentinel-2/data", 'r')
    masks = predictor.predict(arr,bands_indexes={
                "red": 3,
                "green": 2,
                "blue": 1,
                "nir": 7,
                "swir1": 11,
                "swir2": 12,
            }, compute=False)
    masks = np.array(masks)
    mask_arr = zarr.open_array(
                f"gs://{p_path}/sentinel-2/weak_labels",
                "w",
                shape=masks.shape,
                chunks=(1, 1000, 1000),
                dtype=np.uint8,
            )
    mask_arr[:] = masks

In [ ]:

for mask in m_arr[:10]:
    plt.imshow(mask==1)
    plt.show()